In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import random
import joblib
import os
os.chdir('Resources/')

from custom_tuple_scaler import CustomTupleScaler

In [2]:
df = pd.read_csv('6_CC_Encrypted_Data.csv')

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
0,0
1,1
2,0
3,1
4,0
...,...
4039,1
4040,1
4041,0
4042,0


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

3476488068334709321004428324822234875385570758714 , 24846772438844064578074177402607600227383966714210
11587453737390097227590471271492528351599113734033 , 27773639256068336172113331344507987356100129784375
363472194187901669993758967694323171766562404977 , 435807668318848180298083450231339819189020734865
17402451201959742363565894493521834331259341891463 , 15798875891211319973932591331097539913869529544197
85477594117714443314955803616242954325220841032757 , 48132071089559527662724153025685563663794416086559
37951409833662251487848155814881565034966359570932 , 27501205851845249880495748870672033180948245791186
41133527773195702721888254430635588152230146284903 , 25455986176245553370557702814934831441420395444461
52831977755943035119171149659892192776866553558891 , 46342762707402498902325772082906869547404374799532
854204650939784222357072555579547723560401354486 , 42625275023631980949490223498197459029619372792630
10446115961261407535314330840454009363668715221538 , 627021363589623465

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
# df_scaled.to_csv('scaled_new_data.csv', index=False)

f:\Thesis\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
with open("12_SP_Test_Data_RF_IF.txt", "r") as file:
    test_indices = eval(file.read()) 

In [10]:
file_path = "13_CC_F1_Scores_RF_IF.txt"

with open(file_path, "r") as f:
    lines = f.readlines()

reference_f1_scores = []
for line in lines:
    if "F1-score for fold" in line:
        score = float(line.split(":")[1].strip())
        reference_f1_scores.append(score)

In [17]:
fold_to_test = random.randint(1, 10)

test_indices_fold = test_indices[fold_to_test - 1] 
print(f"Test indices for fold {fold_to_test}: {test_indices_fold}")

X_test = df_scaled.iloc[test_indices_fold]

model_path = f'12_SP_Model_RF_IF_{fold_to_test}.joblib'
model = joblib.load(model_path)

Y_test = Y.iloc[test_indices_fold]
Y_pred = model.predict(X_test)

computed_f1_fold = f1_score(Y_test, Y_pred) * 100

reference_f1_fold = reference_f1_scores[fold_to_test - 1]

print(f"Computed F1-score for fold 1: {computed_f1_fold:.2f}")
print(f"Reference F1-score for fold 1: {reference_f1_fold:.2f}")

if np.isclose(computed_f1_fold, reference_f1_fold, atol=1e-2):
    print("The computed F1-score matches the reference score.")
else:
    print("The computed F1-score does not match the reference score.")

Test indices for fold 8: [1, 2, 11, 12, 22, 82, 106, 113, 139, 145, 149, 170, 180, 182, 200, 210, 228, 229, 247, 251, 261, 279, 282, 283, 287, 298, 300, 319, 333, 341, 343, 357, 377, 380, 383, 405, 407, 413, 425, 454, 456, 460, 463, 465, 466, 468, 471, 490, 495, 502, 536, 550, 559, 560, 583, 585, 587, 600, 631, 632, 635, 637, 638, 643, 655, 660, 668, 673, 676, 681, 683, 696, 701, 742, 758, 772, 786, 810, 813, 822, 829, 835, 859, 866, 870, 906, 910, 924, 964, 982, 984, 989, 996, 1002, 1007, 1014, 1019, 1030, 1046, 1048, 1079, 1082, 1086, 1096, 1116, 1139, 1140, 1142, 1153, 1157, 1166, 1167, 1170, 1200, 1205, 1211, 1214, 1215, 1222, 1247, 1251, 1265, 1270, 1295, 1297, 1298, 1337, 1346, 1354, 1356, 1382, 1387, 1394, 1397, 1412, 1415, 1416, 1440, 1442, 1467, 1468, 1489, 1498, 1499, 1511, 1547, 1557, 1581, 1604, 1619, 1627, 1630, 1649, 1656, 1681, 1689, 1694, 1709, 1728, 1735, 1741, 1753, 1757, 1764, 1778, 1785, 1792, 1795, 1808, 1813, 1824, 1833, 1845, 1854, 1856, 1868, 1869, 1870, 1871, 1